In [1]:
# imports
import math
import os
import copy
import json

In [2]:
with open('../scrapedHints.txt', 'r', encoding='utf-8') as file:
    hints = file.readlines()
file.close()
print(len(hints))

35896


In [3]:
import re

only_math_hints = []
for hint in hints:
    strings_between_dollars = re.findall(r'\$\$(.*?)\$\$', hint.replace('\n', ''))

    if strings_between_dollars == -1: # no math in this hint
        only_math_hints.append(' ')
    else:
        strings_between_dollars
        only_math_hints.append('@'.join(strings_between_dollars))

print(len(only_math_hints), only_math_hints)

35896 ['', '4@x', '4', '', '', '-2y@4y^2', '-2y@3y', '-2y', '', '', 'p@x', 'p', '', '', 'p@x', 'p', '', '', 'p', 'p', '', '', '4x^2@2x^2', '4x^2@-3x', '4x^2', '', '', '4x@3x^2', '4x@-5x', '4x', '', '', '4', '4', '', '', '6@b', '6', '', '', '-3', '-3', '', '', '-5@p', '-5', '', '', '5@x', '5', '', '', '2@x', '2', '', '', '7@y', '7', '', '', '-3', '-3', '', '', '-8', '-8', '', '', '', '', '', '', '', '', '', '', '-b@b', '-b', '', '', '-y@y', '-y', '', '', '-x@x', '-x', '', '', '-p@p', '-p', '', '', '3@y', '3', '', '', '6r@4r', '6r', '', '', 'y@y', 'y', '', '', 'x@x', 'x', '', '', 'd@d', 'd', '', '', '7x@2x', '7x@y', '', '', '5x@x', '5x@4y', '', '', '2p@6p', '2p@r', '', '', '', '', '', '', 'n', 'n+1', 'n+2', '', '', '', '', '11@3', '', '11=3+2b', '', '', '8=2b', '2', '', '', 'n-6=13', '', 'n', '', '', '', '', '', '120=\\frac{2}{3} n', '', '\\frac{120}{\\frac{2}{3}}', '120\\frac{3}{2}', '', '', '', '', '', '', '', '\\frac{16}{\\frac{2}{5}}', '16\\frac{5}{2}', '', '', '', '3', '', '7=3+2n',

### Partition into chunks

In [62]:
elements = len(only_math_hints)
chunk_size = 2000

n_lists = elements // chunk_size
remaining_elements = elements % chunk_size

hint_lists = [only_math_hints[i*chunk_size:(i+1)*chunk_size] for i in range(n_lists)]  # partition all but last elements
if remaining_elements > 0:
    hint_lists.append(only_math_hints[n_lists*chunk_size:])    # add last elements

# Print the number of hint_lists and their lengths
print(f"Number of hint_lists: {len(hint_lists)}")
for i, lst in enumerate(hint_lists):
    print(f"List {i+1} length: {len(lst)}")

Number of hint_lists: 18
List 1 length: 2000
List 2 length: 2000
List 3 length: 2000
List 4 length: 2000
List 5 length: 2000
List 6 length: 2000
List 7 length: 2000
List 8 length: 2000
List 9 length: 2000
List 10 length: 2000
List 11 length: 2000
List 12 length: 2000
List 13 length: 2000
List 14 length: 2000
List 15 length: 2000
List 16 length: 2000
List 17 length: 2000
List 18 length: 1896


In [63]:
hint_to_file = []
for hint_list in hint_lists:
    elements = len(hint_list)
    inner_chunk_size = 300

    n_lists = elements // inner_chunk_size
    remaining_elements = elements % inner_chunk_size

    hint_lists_final = [hint_list[i*inner_chunk_size:(i+1)*inner_chunk_size] for i in range(n_lists)]  # partition all but last elements
    if remaining_elements > 0:
        hint_lists_final.append(hint_list[n_lists*inner_chunk_size:])    # add last elements

    hint_to_file.append(hint_lists_final)
    # Print the number of hint_lists and their lengths
    print(f"Number of hint_lists: {len(hint_lists_final)}")
    for i, lst in enumerate(hint_lists_final):
        print(f"List {i+1} length: {len(lst)}")

Number of hint_lists: 7
List 1 length: 300
List 2 length: 300
List 3 length: 300
List 4 length: 300
List 5 length: 300
List 6 length: 300
List 7 length: 200
Number of hint_lists: 7
List 1 length: 300
List 2 length: 300
List 3 length: 300
List 4 length: 300
List 5 length: 300
List 6 length: 300
List 7 length: 200
Number of hint_lists: 7
List 1 length: 300
List 2 length: 300
List 3 length: 300
List 4 length: 300
List 5 length: 300
List 6 length: 300
List 7 length: 200
Number of hint_lists: 7
List 1 length: 300
List 2 length: 300
List 3 length: 300
List 4 length: 300
List 5 length: 300
List 6 length: 300
List 7 length: 200
Number of hint_lists: 7
List 1 length: 300
List 2 length: 300
List 3 length: 300
List 4 length: 300
List 5 length: 300
List 6 length: 300
List 7 length: 200
Number of hint_lists: 7
List 1 length: 300
List 2 length: 300
List 3 length: 300
List 4 length: 300
List 5 length: 300
List 6 length: 300
List 7 length: 200
Number of hint_lists: 7
List 1 length: 300
List 2 length: 

In [87]:
len(hint_to_file), len(hint_to_file[0]), len(hint_to_file[0][0]) # 19 mappar, 100 expressins in each and 1 elements in each hint (joint by @)

(18, 7, 300)

#### Make the files for the text chunks

In [89]:
stop_word = " nyradhej "

for i in range(len(hint_to_file)): # 0-18
    directory_name = f"math-conversion{i}" # man kan inte skapa nytt directory så detta borde vara fine att inte skriva över existerande
    os.mkdir(directory_name) # should throw an error if you have already created this folder because you should not overwrite the first creation 

    for c, hint_chunk in enumerate(hint_to_file[i]):
        filename = f"{directory_name}/onlyMath{i}-{c}Pre.txt"
        
        with open(filename, 'w',  encoding='utf-8') as file:
            for math_hint in hint_chunk:
                file.write(math_hint + ';')
            file.write(stop_word)
        file.close()


        empty_filename = f"{directory_name}/onlyMath{i}-{c}Post.txt"
        with open(empty_filename, 'w',  encoding='utf-8') as empty_file:
            pass
        empty_file.close()
    

### Collect mts translations and put into one file

In [171]:
# run after you have pasted all the chunks in folder i from MathCAT
# i++ if you want to create next summary of translation
folder_index = 5
directory_name = f"math-conversion{folder_index}"
collective_filename = f"post-{folder_index}"
write_filepath = f"{directory_name}/{collective_filename}.txt"

if os.path.isfile(write_filepath):
  print(f"File {collective_filename} already exists, delete it first if you want to rewrite it") # to not write over a file we want to keep
else:
  # create new file to collect the 
  with open(write_filepath, 'w',  encoding='utf-8') as empty_file:
              pass
  empty_file.close()

  with open(write_filepath, 'a',  encoding='utf-8') as write_file:              # opens file to append 

    for c in range(7):                                                          # for text-chunk (c) in dir i                  

      read_filepath = f"{directory_name}/onlyMath{folder_index}-{c}Post.txt"              

      with open(read_filepath, 'r',  encoding='utf-8') as read_file:          # read from subfile
          read_info = read_file.readlines()
          write_file.writelines(read_info)
      read_file.close()
  write_file.close()
  print(f"File {collective_filename} have been created with all converted latex from {directory_name}")
    

File post-5 have been created with all converted latex from math-conversion5


#### Put math back in full hints

In [175]:
# all the converted latex-to-speech have been collected in the post-i file and will later be inserted into the full hints
# here the mts is read 
# i = 0
# directory_name = f"math-conversion{folder_index}"
# collective_filename = f"post-{folder_index}"
all_math_filepath = write_filepath #f"{directory_name}/{collective_filename}.txt"

with open(all_math_filepath, 'r') as file:
    math = file.readline()
file.close()

math = math.split('semicolon')
math = [hint.split('at sign') for hint in math]

len(math), math

(2001,
 [['b '],
  ['; 2 x minus 3 y; is equal to, 5 ',
   ' negative 3 y; is equal to; negative 2 x, plus; 5 ',
   '; fraction, negative 3 y, over, negative 3, end fraction; is equal to; fraction, open paren, negative 2 x, plus 5, close paren, over, negative 3, end fraction; ',
   ' y; is equal to, 2 thirds x minus 5 thirds; '],
  [' m '],
  [' b; '],
  ['; '],
  [' 1 '],
  [' 1; '],
  ['; '],
  ['; y is equal to; fraction, negative 1, over 2, end fraction; x; plus 5; '],
  [' m '],
  [' b '],
  ['; x plus 2 y, is equal to, 10 ',
   ' 2 y, is equal to; negative x plus; 10 ',
   ', fraction, 2 y, over 2, end fraction; is equal to; fraction, open paren, negative x plus 10, close paren, over 2, end fraction; ',
   ' y; is equal to; fraction, negative 1, over 2, end fraction; x; plus 5; '],
  [' m '],
  [' b; '],
  ['; '],
  ['; '],
  ['; '],
  ['; '],
  [', y is equal to x plus 1; '],
  [' m '],
  [' b; '],
  ['; minus x plus y; is equal to 1 ', ' y, is equal to x plus 1; '],
  [' m '],


In [176]:
chunk_size = 2000
hints_done = copy.deepcopy(hints)
start = chunk_size * folder_index
stop = start + len(hint_lists[folder_index]) # -1
j = 0

for i in range(start, stop):
    math_replacements = math[j]
    for replacement in math_replacements:
        hints_done[i]  = re.sub(r'\$\$.*?\$\$', replacement, hints_done[i], count=1)
    j +=1

translated_dir = hints_done[start:stop]
print(f"Check scrapedHints {start+1} - {stop}")
len(translated_dir), translated_dir

Check scrapedHints 10001 - 12000


(2000,
 ['What is b  in the equation?\n',
  'Write the second equation in slope-intercept form:\\n; 2 x minus 3 y; is equal to, 5 \\n negative 3 y; is equal to; negative 2 x, plus; 5 \\n; fraction, negative 3 y, over, negative 3, end fraction; is equal to; fraction, open paren, negative 2 x, plus 5, close paren, over, negative 3, end fraction; \\n y; is equal to, 2 thirds x minus 5 thirds; \n',
  'What is  m  in the equation?\n',
  'What is  b;  in the equation?\n',
  'Since the slopes are different, the lines intersect.\n',
  'A system of equations whose graphs are intersect has  1  solution.\n',
  'A system of equations whose graphs are intersect has  1;  solution and is consistent and independent.\n',
  'We will compare the slopes and intercepts of the two lines: image1\n',
  'The first equation is already in slope-intercept form: ; y is equal to; fraction, negative 1, over 2, end fraction; x; plus 5; \n',
  'What is  m  in the equation?\n',
  'What is  b  in the equation?\n',
  'Wr

In [177]:
finished_file = f"finishedHints-{folder_index}.txt"

with open(finished_file, 'w', encoding='utf-8') as file:
    for hint in translated_dir: 
        file.write(hint)
file.close()

### Collect all files thus far

In [195]:
n_folders_thus = 6
file_names = [f'finishedHints-{finished_index}.txt' for finished_index in range(n_folders_thus)]
file_names

['finishedHints-0.txt',
 'finishedHints-1.txt',
 'finishedHints-2.txt',
 'finishedHints-3.txt',
 'finishedHints-4.txt',
 'finishedHints-5.txt']

In [197]:
# combine all translated hints
combined_file_name = 'combined_finishedHints.txt'

# Open the combined text file in write mode
with open(combined_file_name, 'w', encoding='utf-8') as combined_file:
    # Iterate over each file name
    for file_name in file_names:
        # Open each file in read mode
        with open(file_name, 'r', encoding='utf-8') as file:
            # Read the contents of the file
            contents = file.read()
            # Write the contents to the combined file
            combined_file.write(contents)
        # Close the file
        file.close()

# Close the combined file
combined_file.close()

print(f"All files have been combined into {combined_file_name}.")

All files have been combined into combined_finishedHints.txt.


In [5]:
only_math_hints[1]

'4@x'

In [9]:
math_filename = 'math.txt'
with open(math_filename, 'w', encoding='utf-8') as file:
    for math_line in only_math_hints:
        file.write(math_line + '\n')
file.close()

### Write to json

In [183]:
for hint in only_math_hints:
    if hint == '':
        print("None here")
    else:
        print(hint)

None here
4@x
4
None here
None here
-2y@4y^2
-2y@3y
-2y
None here
None here
p@x
p
None here
None here
p@x
p
None here
None here
p
p
None here
None here
4x^2@2x^2
4x^2@-3x
4x^2
None here
None here
4x@3x^2
4x@-5x
4x
None here
None here
4
4
None here
None here
6@b
6
None here
None here
-3
-3
None here
None here
-5@p
-5
None here
None here
5@x
5
None here
None here
2@x
2
None here
None here
7@y
7
None here
None here
-3
-3
None here
None here
-8
-8
None here
None here
None here
None here
None here
None here
None here
None here
None here
None here
-b@b
-b
None here
None here
-y@y
-y
None here
None here
-x@x
-x
None here
None here
-p@p
-p
None here
None here
3@y
3
None here
None here
6r@4r
6r
None here
None here
y@y
y
None here
None here
x@x
x
None here
None here
d@d
d
None here
None here
7x@2x
7x@y
None here
None here
5x@x
5x@4y
None here
None here
2p@6p
2p@r
None here
None here
None here
None here
None here
None here
n
n+1
n+2
None here
None here
None here
None here
11@3
None here
11=3+2b
N

In [198]:
# collect all json filepaths to write to
filepaths = []
for root, dirs, files in os.walk('src\a-math-to-speech\content-pool-mts'): 
    for file in files:
        if 'DefaultPathway' in file:
            filepaths.append(os.path.join(root, file))
filepaths

[]

In [ ]:
# Read the contents of the finishedHints-{i}.txt files

with open('src/a-math-to-speech/finishedHints.txt', 'r') as file:
    hints = file.readlines()
    hints = [hint.strip() for hint in hints]
file.close()

In [ ]:
hint_intex = 0
for source_file in filepaths:
    # Read the contents of the source file

    with open(source_file, 'r+') as file:
        data = json.load(file)

        # Update the "text" attribute with hints[i]
        for obj in data:
            
            obj['speech'] = hints[hint_intex]

            hint_intex+=1
       
        # Write the updated JSON object to the source file
        file.seek(0)
        file.write(json.dumps(data, indent=4))
        file.truncate()

    file.close()
        #contents = file.read()
#         data = json.load(file)

#         # Extract the "text" attribute from each JSON object
#         contents = [obj['text'] for obj in data]

#         # Write the contents to the output file
#         for content in contents:
#             outfile.write(content + "\n")

print(f"Contents have been written")
